In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
MODEL_NAME = "distilbert/distilbert-base-multilingual-cased"
from bert_lightning import songs_from_artists
from sklearn.model_selection import train_test_split
from utils import get_all_artists, split_into_year_buckets
from datasets import Dataset
import pandas as pd
from transformers import pipeline
import shap

2024-07-01 15:14:05.296315: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-01 15:14:05.327151: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 15:14:05.968562: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model_loaded = AutoModelForSequenceClassification.from_pretrained("year_bert_classifier")

In [8]:
arts = get_all_artists()
label2art = {label: i for i, label in enumerate((a.name_sanitized for a in arts))}
art2label = {i: label for label, i in label2art.items()}
arts = get_all_artists()
data = songs_from_artists(arts, tokenizer, label2id=label2art, mode='solo', song_limit=190)

df = pd.DataFrame(data)
df['label'] = split_into_year_buckets(df['year'])
df = df[df['year'].between(1985, 2025)]
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42)
texts = [tokenizer.decode(tokens, skip_special_tokens=True) for tokens in test_df["input_ids"]]
texts_dataset = Dataset.from_dict({"text":texts})

100%|██████████| 1049/1049 [00:31<00:00, 33.40it/s]


In [20]:
pred = pipeline(
    "text-classification",
    model=model_loaded,
    tokenizer=tokenizer, 
    return_all_scores=True,
    top_k=None,
    padding = True,
    truncation = True,
    device='cuda',
    batch_size=8,
    )
explainer = shap.Explainer(pred)


In [22]:
len(texts_dataset)

5040

In [21]:
shap_values = explainer(texts_dataset[:1008])
import pickle
with open("shap_values_years_big.pkl", "wb") as f:
    pickle.dump(shap_values, f)

PartitionExplainer explainer: 17it [01:07,  4.85s/it]


KeyboardInterrupt: 

In [11]:
shap.plots.text(shap_values[0])